In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from mpl_toolkits import mplot3d
sns.set()

/home/jhallbe/anaconda3/envs/waldo/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/jhallbe/anaconda3/envs/waldo/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/jhallbe/anaconda3/envs/waldo/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/jhallbe/anaconda3/envs/waldo/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/jhallbe/anaconda3/envs/waldo/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(

# Load Data

In [168]:
def load_data(data_location):
    return pd.read_csv(data_location)

In [169]:
data_location = "data/denver_data.csv"
df = load_data(data_location)

In [170]:
df.head()

,datetime,humidity_Denver,pressure_Denver,temperature_Denver,weather_description_Denver,wind_direction_Denver,wind_speed_Denver
0,2012-10-01 13:00:00,62.0,1028.0,284.610000,light rain,20.0,4.0
1,2012-10-01 14:00:00,62.0,1028.0,284.607306,broken clouds,22.0,4.0
2,2012-10-01 15:00:00,62.0,1028.0,284.599918,broken clouds,31.0,3.0
3,2012-10-01 16:00:00,62.0,1028.0,284.592530,broken clouds,39.0,3.0
4,2012-10-01 17:00:00,63.0,1028.0,284.585142,broken clouds,47.0,3.0


# Create Series Data

In [183]:
def find_unique_year_month_day(data):
    unique_year_month_day = []
    
    for date in data["datetime"]:
        year_month_day = date.split(" ")[0]
        if year_month_day not in unique_year_month_day:
            unique_year_month_day.append(year_month_day)

    return unique_year_month_day
        
def fill_missing_values(data):
    year_month_day = data["datetime"].values[0].split(" ")[0]
    
    hours = []
    for i in range(24):
        if i < 10: hour = "0" + str(i)
        else: hour = str(i)
            
        hours.append(year_month_day + " " + hour + ":00:00")
    
    return data.set_index('datetime').reindex(hours).fillna(0).reset_index()
        
    
def series(data, time):
    # returns 24 hours of series data for a given time.
    # time is in the year-month-day format with no hours.
    time_series = df.loc[df['datetime'].str.startswith(time)]
    time_series = fill_missing_values(time_series)
    
    return time_series

In [185]:
unique_year_month_day = sorted(find_unique_year_month_day(df))

In [186]:
series_data = pd.DataFrame(columns=df.columns)

for year_month_day in unique_year_month_day:
    series_data = series_data.append(series(df, year_month_day))

In [200]:
series_data.head()

,datetime,humidity_Denver,pressure_Denver,temperature_Denver,weather_description_Denver,wind_direction_Denver,wind_speed_Denver
0,2012-10-01 00:00:00,0.0,0.0,0.0,0,0.0,0.0
1,2012-10-01 01:00:00,0.0,0.0,0.0,0,0.0,0.0
2,2012-10-01 02:00:00,0.0,0.0,0.0,0,0.0,0.0
3,2012-10-01 03:00:00,0.0,0.0,0.0,0,0.0,0.0
4,2012-10-01 04:00:00,0.0,0.0,0.0,0,0.0,0.0


In [213]:
def description_values(data):
    categorical_values = {}
    categorical_value = 1
    for weather_description in data["weather_description_Denver"]:
        if weather_description != '0' and weather_description not in categorical_values:
            categorical_values[weather_description] = categorical_value
            categorical_value += 1
    return categorical_values

In [217]:
categorical_values = description_values(series_data)
categorical_values

{'broken clouds': 2,
 'drizzle': 19,
 'dust': 26,
 'few clouds': 5,
 'fog': 13,
 'haze': 11,
 'heavy intensity drizzle': 32,
 'heavy intensity rain': 18,
 'heavy snow': 9,
 'light intensity drizzle': 16,
 'light intensity shower rain': 23,
 'light rain': 1,
 'light rain and snow': 31,
 'light snow': 14,
 'mist': 10,
 'moderate rain': 21,
 'overcast clouds': 8,
 'proximity shower rain': 3,
 'proximity thunderstorm': 12,
 'proximity thunderstorm with rain': 28,
 'ragged thunderstorm': 34,
 'scattered clouds': 4,
 'shower rain': 29,
 'sky is clear': 6,
 'smoke': 24,
 'snow': 15,
 'squalls': 7,
 'thunderstorm': 20,
 'thunderstorm with heavy drizzle': 33,
 'thunderstorm with heavy rain': 25,
 'thunderstorm with light drizzle': 30,
 'thunderstorm with light rain': 22,
 'thunderstorm with rain': 17,
 'very heavy rain': 27}

In [221]:
series_data["weather_description_Denver"].replace(categorical_values, inplace=True)

In [222]:
series_data.to_csv("data/denver_series_data.csv", index=False)